# Market Basket Analysis - Memory Efficient Version

This notebook performs market basket analysis on retail data using a memory-efficient approach.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from itertools import combinations
import collections

## 1. Data Loading

In [2]:
# Configuration
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx'
raw_data_path = '../data/01_raw/online_retail_II.xlsx'

# Create directory and download if needed
os.makedirs(os.path.dirname(raw_data_path), exist_ok=True)

if not os.path.exists(raw_data_path):
    print('Downloading dataset...')
    response = requests.get(url)
    response.raise_for_status()
    with open(raw_data_path, 'wb') as f:
        f.write(response.content)
    print('Download complete.')
else:
    print('Dataset already exists.')

Dataset already exists.


In [3]:
# Load data from both sheets
print('Loading data...')
xls = pd.ExcelFile(raw_data_path)
df_year_1 = pd.read_excel(xls, 'Year 2009-2010')
df_year_2 = pd.read_excel(xls, 'Year 2010-2011')

# Combine datasets
df = pd.concat([df_year_1, df_year_2], ignore_index=True)
print(f'Total data shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
df.head()

Loading data...
Total data shape: (1067371, 8)
Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 2. Data Preprocessing & Sampling

In [4]:
# Basic cleaning
print('Original shape:', df.shape)

# Remove rows with missing CustomerID
df = df.dropna(subset=['Customer ID'])
print('After removing null CustomerID:', df.shape)

# Remove returns (invoices starting with 'C')
df = df[~df['Invoice'].astype(str).str.startswith('C')]
print('After removing returns:', df.shape)

# Keep only positive quantities
df = df[df['Quantity'] > 0]
print('After removing negative quantities:', df.shape)

# Clean descriptions
df['Description'] = df['Description'].str.strip()
df = df.dropna(subset=['Description'])
print('After cleaning descriptions:', df.shape)

Original shape: (1067371, 8)
After removing null CustomerID: (824364, 8)
After removing returns: (805620, 8)
After removing negative quantities: (805620, 8)
After cleaning descriptions: (805620, 8)


In [5]:
# ULTRA-CONSERVATIVE SAMPLING for guaranteed success
print('\n=== SMART SAMPLING FOR MEMORY EFFICIENCY ===')

# Step 1: Take only top 50 most popular products
product_counts = df['Description'].value_counts()
top_products = product_counts.head(50).index.tolist()

print(f'Top 50 products:')
for i, (product, count) in enumerate(product_counts.head(50).items(), 1):
    print(f'{i:2d}. {product[:50]:<50} ({count:,} times)')

# Filter to only these products
df_sample = df[df['Description'].isin(top_products)].copy()
print(f'\nData with top 50 products: {df_sample.shape}')

# Step 2: Sample 10000 random invoices
unique_invoices = df_sample['Invoice'].unique()
sample_invoices = np.random.choice(unique_invoices, size=min(10000, len(unique_invoices)), replace=False)
df_final = df_sample[df_sample['Invoice'].isin(sample_invoices)].copy()

print(f'\nFinal sample: {df_final.shape}')
print(f'Unique invoices: {df_final["Invoice"].nunique()}')
print(f'Unique products: {df_final["Description"].nunique()}')



=== SMART SAMPLING FOR MEMORY EFFICIENCY ===
Top 50 products:
 1. WHITE HANGING HEART T-LIGHT HOLDER                 (5,181 times)
 2. REGENCY CAKESTAND 3 TIER                           (3,430 times)
 3. ASSORTED COLOUR BIRD ORNAMENT                      (2,777 times)
 4. JUMBO BAG RED RETROSPOT                            (2,702 times)
 5. REX CASH+CARRY JUMBO SHOPPER                       (2,141 times)
 6. PARTY BUNTING                                      (2,122 times)
 7. LUNCH BAG  BLACK SKULL.                            (2,117 times)
 8. LUNCH BAG SPACEBOY DESIGN                          (1,941 times)
 9. HOME BUILDING BLOCK WORD                           (1,929 times)
10. STRAWBERRY CERAMIC TRINKET BOX                     (1,922 times)
11. HEART OF WICKER SMALL                              (1,899 times)
12. 60 TEATIME FAIRY CAKE CASES                        (1,880 times)
13. HEART OF WICKER LARGE                              (1,864 times)
14. LUNCH BAG CARS BLUE                 

## 3. Market Basket Analysis

In [6]:
# Simple co-occurrence analysis (no memory issues)
print('\n=== MARKET BASKET ANALYSIS ===')

# Group products by invoice
invoice_products = df_final.groupby('Invoice')['Description'].apply(list).reset_index()
print(f'Analyzing {len(invoice_products)} invoices...')

# Count co-occurrences
pair_counts = collections.Counter()
product_counts = collections.Counter()
total_baskets = 0

for products in invoice_products['Description']:
    if len(products) >= 2:  # Only multi-item baskets
        total_baskets += 1
        
        # Count individual products
        for product in products:
            product_counts[product] += 1
        
        # Count pairs
        for pair in combinations(sorted(products), 2):
            pair_counts[pair] += 1

print(f'Multi-item baskets: {total_baskets}')
print(f'Unique product pairs: {len(pair_counts)}')


=== MARKET BASKET ANALYSIS ===
Analyzing 10000 invoices...
Multi-item baskets: 7112
Unique product pairs: 1272


In [7]:
# Calculate association metrics
associations = []

for (product_a, product_b), pair_count in pair_counts.items():
    # Support: P(A and B)
    support = pair_count / total_baskets
    
    # Confidence: P(B|A) and P(A|B)
    confidence_a_to_b = pair_count / product_counts[product_a]
    confidence_b_to_a = pair_count / product_counts[product_b]
    
    # Lift: Support / (P(A) * P(B))
    prob_a = product_counts[product_a] / total_baskets
    prob_b = product_counts[product_b] / total_baskets
    lift = support / (prob_a * prob_b)
    
    associations.append({
        'Product_A': product_a,
        'Product_B': product_b,
        'Count': pair_count,
        'Support': support,
        'Confidence_A→B': confidence_a_to_b,
        'Confidence_B→A': confidence_b_to_a,
        'Lift': lift
    })

# Create results DataFrame
results_df = pd.DataFrame(associations)
results_df = results_df.sort_values(['Lift', 'Support'], ascending=[False, False])

print(f'\nTotal associations found: {len(results_df)}')
print('\nTop 15 Product Associations:')
display(results_df.head(15).round(3))


Total associations found: 1272

Top 15 Product Associations:


,Product_A,Product_B,Count,Support,Confidence_A→B,Confidence_B→A,Lift
74,STRAWBERRY CERAMIC TRINKET BOX,SWEETHEART CERAMIC TRINKET BOX,389,0.055,0.575,0.842,8.845
1072,PAPER CHAIN KIT 50'S CHRISTMAS,PAPER CHAIN KIT VINTAGE CHRISTMAS,305,0.043,0.504,0.695,8.167
277,CHOCOLATE HOT WATER BOTTLE,HOT WATER BOTTLE TEA AND SYMPATHY,283,0.040,0.519,0.576,7.521
93,WOODEN FRAME ANTIQUE WHITE,WOODEN PICTURE FRAME WHITE FINISH,443,0.062,0.672,0.680,7.344
25,COOK WITH WINE METAL SIGN,GIN + TONIC DIET METAL SIGN,254,0.036,0.513,0.502,7.212
26,COOK WITH WINE METAL SIGN,HAND OVER THE CHOCOLATE SIGN,230,0.032,0.465,0.451,6.480
578,COOK WITH WINE METAL SIGN,PLEASE ONE PERSON METAL SIGN,238,0.033,0.481,0.445,6.392
855,HEART OF WICKER LARGE,HEART OF WICKER SMALL,411,0.058,0.623,0.588,6.336
30,GIN + TONIC DIET METAL SIGN,HAND OVER THE CHOCOLATE SIGN,222,0.031,0.439,0.435,6.118
114,60 TEATIME FAIRY CAKE CASES,72 SWEETHEART FAIRY CAKE CASES,257,0.036,0.376,0.581,6.055


In [19]:
# --- Save the rules DataFrame to a file ---

# We will use pickle to save our rules object for later use in the API.
import pickle

# Define the path to save the model/rules
model_path = '../models/association_rules.pkl'

# Save the DataFrame to a pickle file
with open(model_path, 'wb') as f:
    pickle.dump(results_df, f)

print(f"Rules saved successfully to {model_path}")

Rules saved successfully to ../models/association_rules.pkl


## 4. Business Insights

In [8]:
# Filter for strong associations
strong_associations = results_df[
    (results_df['Support'] >= 0.01) &  # At least 1% support
    (results_df['Lift'] >= 1.5)        # At least 50% lift
]

print('\n' + '='*60)
print('BUSINESS INSIGHTS')
print('='*60)

if len(strong_associations) > 0:
    print(f'\n🎯 STRONG ASSOCIATIONS FOUND: {len(strong_associations)}')
    print('\nTop recommendations for cross-selling:')
    
    for i, row in strong_associations.head(10).iterrows():
        print(f'\n{len(strong_associations.head(10)) - len(strong_associations.head(10)[strong_associations.head(10).index <= i]) + 1}. "{row["Product_A"]}" + "{row["Product_B"]}"')
        print(f'   • Lift: {row["Lift"]:.2f}x more likely to be bought together')
        print(f'   • Confidence: {max(row["Confidence_A→B"], row["Confidence_B→A"]):.1%} of customers who buy one also buy the other')
        print(f'   • Support: {row["Support"]:.1%} of all transactions contain both items')
    
    print(f'\n📊 SUMMARY STATISTICS:')
    print(f'• Average lift for strong associations: {strong_associations["Lift"].mean():.2f}')
    print(f'• Highest lift found: {strong_associations["Lift"].max():.2f}')
    print(f'• Most frequent pair appears in {strong_associations["Support"].max():.1%} of transactions')
    
else:
    print('\n⚠️  No strong associations found with current thresholds.')
    print('\nTop 5 associations by lift (regardless of thresholds):')
    
    for i, row in results_df.head(5).iterrows():
        print(f'\n{i+1}. "{row["Product_A"]}" + "{row["Product_B"]}"')
        print(f'   • Lift: {row["Lift"]:.2f}')
        print(f'   • Support: {row["Support"]:.1%}')

print(f'\n💡 RECOMMENDATIONS:')
print('• Use these associations for product placement and bundling')
print('• Consider promotional campaigns for high-lift pairs')
print('• Monitor inventory levels for associated products')
print('• Implement "customers who bought X also bought Y" recommendations')


BUSINESS INSIGHTS

🎯 STRONG ASSOCIATIONS FOUND: 149

Top recommendations for cross-selling:

7. "STRAWBERRY CERAMIC TRINKET BOX" + "SWEETHEART CERAMIC TRINKET BOX"
   • Lift: 8.85x more likely to be bought together
   • Confidence: 84.2% of customers who buy one also buy the other
   • Support: 5.5% of all transactions contain both items

1. "PAPER CHAIN KIT 50'S CHRISTMAS" + "PAPER CHAIN KIT VINTAGE CHRISTMAS"
   • Lift: 8.17x more likely to be bought together
   • Confidence: 69.5% of customers who buy one also buy the other
   • Support: 4.3% of all transactions contain both items

4. "CHOCOLATE HOT WATER BOTTLE" + "HOT WATER BOTTLE TEA AND SYMPATHY"
   • Lift: 7.52x more likely to be bought together
   • Confidence: 57.6% of customers who buy one also buy the other
   • Support: 4.0% of all transactions contain both items

6. "WOODEN FRAME ANTIQUE WHITE" + "WOODEN PICTURE FRAME WHITE FINISH"
   • Lift: 7.34x more likely to be bought together
   • Confidence: 68.0% of customers who

In [9]:
# Save results
output_path = '../data/02_processed/market_basket_results.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
results_df.to_csv(output_path, index=False)
print(f'\nResults saved to: {output_path}')
print(f'Total associations: {len(results_df)}')
print('\nAnalysis complete! ✅')


Results saved to: ../data/02_processed/market_basket_results.csv
Total associations: 1272

Analysis complete! ✅


## 5. Recomendation System

In [16]:
def get_recommendations(product, results_df, top_n=5, min_lift=1.0):
    """
    Given a product name, return top N recommended products based on association rules.
    
    Args:
        product (str): Product name to get recommendations for.
        results_df (pd.DataFrame): DataFrame with association rules.
        top_n (int): Number of recommendations to return.
        min_lift (float): Minimum lift threshold for recommendations.
        
    Returns:
        pd.DataFrame: Top N recommended products with metrics.
    """
    # Find associations where the product is either Product_A or Product_B
    mask_a = (results_df['Product_A'] == product) & (results_df['Lift'] >= min_lift)
    mask_b = (results_df['Product_B'] == product) & (results_df['Lift'] >= min_lift)
    
    recs_a = results_df[mask_a].copy()
    recs_a['Recommended_Product'] = recs_a['Product_B']
    recs_a['Direction'] = 'A→B'
    recs_a['Confidence'] = recs_a['Confidence_A→B']
    
    recs_b = results_df[mask_b].copy()
    recs_b['Recommended_Product'] = recs_b['Product_A']
    recs_b['Direction'] = 'B→A'
    recs_b['Confidence'] = recs_b['Confidence_B→A']
    
    recs = pd.concat([recs_a, recs_b], ignore_index=True)
    recs = recs.sort_values(['Lift', 'Confidence', 'Support'], ascending=[False, False, False])
    
    # Select columns to display
    cols = ['Recommended_Product', 'Support', 'Confidence', 'Lift', 'Count', 'Direction']
    return recs[cols].head(top_n).reset_index(drop=True)

In [18]:
# Test get_recommendations function on 'ALARM CLOCK BAKELIKE GREEN'
test_product = 'STRAWBERRY CERAMIC TRINKET BOX'
recommendations = get_recommendations(test_product, results_df)
display(recommendations)

,Recommended_Product,Support,Confidence,Lift,Count,Direction
0,SWEETHEART CERAMIC TRINKET BOX,0.054696,0.574594,8.845262,389,A→B
1,60 TEATIME FAIRY CAKE CASES,0.019544,0.205318,2.137948,139,B→A
2,72 SWEETHEART FAIRY CAKE CASES,0.010827,0.113737,1.830086,77,B→A
3,GIN + TONIC DIET METAL SIGN,0.010264,0.107829,1.515568,73,B→A
4,PINK CREAM FELT CRAFT TRINKET BOX,0.008577,0.090103,1.405297,61,B→A
